In [1]:
!pip install numpy pandas scikit-learn scikit-image matplotlib tqdm joblib

In [1]:
import numpy as np
import os
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tqdm import tqdm
import joblib

# Optimized Configuration
IMG_SIZE = (128, 128)          # Balanced resolution
MAX_SAMPLES = 1200             # Sufficient data
TEST_SIZE = 0.2
RANDOM_STATE = 42
ORIENTATIONS = 9               # HOG parameters
PIXELS_PER_CELL = (16, 16)     # HOG parameters

def extract_hog_features(images):
    """Enhanced HOG feature extraction"""
    features = []
    for img in tqdm(images, desc="Extracting HOG"):
        fd = hog(img,
                orientations=ORIENTATIONS,
                pixels_per_cell=PIXELS_PER_CELL,
                cells_per_block=(2, 2),
                channel_axis=-1)
        features.append(fd)
    return np.array(features)

def load_images(root_dir):
    """Efficient image loader"""
    images = []
    labels = []
    
    for class_idx, class_name in enumerate(['Cat', 'Dog']):
        class_dir = os.path.join(root_dir, class_name)
        print(f"Loading {class_name} images...")
        
        files = [f for f in os.listdir(class_dir) 
                if f.lower().endswith(('.jpg','.jpeg','.png'))][:MAX_SAMPLES]
        
        for filename in tqdm(files):
            try:
                img = imread(os.path.join(class_dir, filename))
                img = resize(img, IMG_SIZE)
                if len(img.shape) == 2:
                    img = np.stack((img,)*3, axis=-1)
                images.append(img)
                labels.append(class_idx)
            except:
                continue
                
    return np.array(images), np.array(labels)

def main():
    # 1. Load data
    print("Loading dataset...")
    X, y = load_images(r"C:\Users\Mohammad Rihan\Downloads\PetImages")
    print(f"\nLoaded {len(X)} images")

    # 2. Extract HOG features
    X_features = extract_hog_features(X)

    # 3. Train/test split
    X_train, X_test, y_train, y_test = train_test_split(
        X_features, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y)

    # 4. Feature scaling
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # 5. Optimized SVM training
    print("\nTraining SVM with RBF kernel...")
    param_grid = {
        'C': [0.1, 1, 10, 100],      # Wider range
        'gamma': ['scale', 0.1, 0.01, 0.001],
        'kernel': ['rbf']
    }
    
    svm_model = svm.SVC(class_weight='balanced', random_state=RANDOM_STATE)
    clf = GridSearchCV(svm_model, param_grid, cv=3, n_jobs=-1, verbose=2)
    clf.fit(X_train, y_train)

    # 6. Evaluation
    y_pred = clf.predict(X_test)
    print("\nBest Parameters:", clf.best_params_)
    print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("\nClassification Report:\n", classification_report(y_test, y_pred))
    print(f"\nAccuracy: {accuracy_score(y_test, y_pred):.2%}")

    # 7. Save model
    joblib.dump({
        'model': clf.best_estimator_,
        'scaler': scaler,
        'hog_params': {
            'orientations': ORIENTATIONS,
            'pixels_per_cell': PIXELS_PER_CELL
        }
    }, 'optimized_svm_model.pkl')

if __name__ == "__main__":
    main()

Loading dataset...
Loading Cat images...


100%|██████████████████████████████████████████████████████████████████████████████| 1200/1200 [00:34<00:00, 34.48it/s]


Loading Dog images...


100%|██████████████████████████████████████████████████████████████████████████████| 1200/1200 [00:34<00:00, 35.28it/s]



Loaded 2395 images


Extracting HOG: 100%|█████████████████████████████████████████████████████████████| 2395/2395 [00:12<00:00, 184.40it/s]



Training SVM with RBF kernel...
Fitting 3 folds for each of 16 candidates, totalling 48 fits

Best Parameters: {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}

Confusion Matrix:
 [[194  45]
 [ 76 164]]

Classification Report:
               precision    recall  f1-score   support

           0       0.72      0.81      0.76       239
           1       0.78      0.68      0.73       240

    accuracy                           0.75       479
   macro avg       0.75      0.75      0.75       479
weighted avg       0.75      0.75      0.75       479


Accuracy: 74.74%
